In [2]:
import pandas as pd
import numpy as np
from pathlib import Path
import pyreadstat

In [3]:
#- Define directories and output path
Raw_Dir=Path("F:\\OneDrive\\Clinical Programming\\Data for Practice\\SDTM Dataset 2\\raw\\raw")
STUDYID="CAEL-101-301"
LB_OUT_PATH=Path("lb_out2.csv")

In [4]:
# Define Helper Functiions-

#1-Date Parser = Parse a date/time value and return ISO datetime string 'YYYY-MM-DDTHH:MM:SS' or pd.NA if parsing fails
def to_iso(x):
    t=pd.to_datetime(x,errors="coerce")
    return t.strftime("%Y-%m-%DT%H:%M:%S")
    

In [5]:
#2-First col-  Return the first matching column name in df (case-insensitive). "Given a list of possible names, return the actual column name present."
def first_col(df,candidates):
    cols=[c.lower() for c in df.columns]
    for cand in candidates:
        if cand.lower() in cols:
            return next(c for c in df.columns if cand.lower()==c.lower())
    return None

In [6]:
#3-Try Num- Convert x to float safely. Returns np.nan if conversion fails or x is missing. "Try to make this value a number; otherwise mark it missing."
def try_num(x):
    try:
        return float(str(x).replace({",",""})) if not pd.isna(x) else np.nan
    except:
        return np.nan

In [7]:
#4- #Compute study day as (lab_date - reference_date).days + 1 Accepts ISO strings or pd.NA. Returns pd.NA when not computable.  "How many days since the study start (Day 1 = RFSTDTC)."
def lb_day(event_iso,reference_iso):
    e=event.apply(to_iso) if pd.notna(event) else np.nan
    r=reference.apply(to_iso) if pd.notna(reference) else np.nan
    return (int(e-r).days)+1

In [8]:
#5-Collect Dates-    """Build list of (SUBJID, DATE) for any detected date-like columns in df.Used to derive RFSTDTC / RFENDTC ranges per subject. """
def collect_dates(df):
    rows=[]
    if df is None :
        return []
    subj_col=first_col(df,["subjectid","subjid","subid","subno"])
    if subj_col is None:
        return []
    date_col=[c for c in df.columns if any(k in c.lower() for k in ["date","dtc","dthc","start","end","dttc","startdate"])]
    if not date_col:
        return []
    for _,r in df.iterrows():
        raw_subj=r.get(subj_col,None)
        if pd.isna(raw_subj):
            continue
        subj=str(raw_subj)
        if not subj:
            continue
        for dc in date_col:
            date=r.get(dc,None)
            if pd.isna(date):
                continue
            parsed=to_iso(date)
            if not parsed:
                continue
            rows.append({"SUBJID":subj,"DATE":parsed})
    return rows

In [9]:
# ---------- Load datasets ----------(Use the names upper case or lowercase based on the datasets named in the directory)

DATASETS=["dm","ae","lb","ex","sv"]

In [10]:
#Create a dictionary of datasets to pull datasets by calling out the name

datasets={}

for name in DATASETS:
    p= Raw_Dir/f"{name}.sas7bdat"
    df,_=pyreadstat.read_sas7bdat(str(p))
    df.columns=[str(c).strip() for c in df.columns]
    for cols in df.select_dtypes(include=object):
        df[cols]=df[cols].replace({"":pd.NA})
        datasets[name]=df

In [11]:
dm_raw=datasets["dm"]
lb_raw=datasets["lb"]
ae_raw=datasets["ae"]
visits_raw=datasets["sv"]
ex_raw=datasets["ex"]

In [12]:
# Build Subjects dataset to get unique subjects do with the help of DM(demographics) data set-

subjects=pd.DataFrame()
subj_col=first_col(dm_raw,["subjectid","subjectno","subjid","subid"])
subjects["SUBJID"]=dm_raw[subj_col]

In [13]:
subjects["STUDYID"]= STUDYID

In [14]:
subjects["USUBJID"]=STUDYID+"-"+ subjects["SUBJID"]


In [15]:
# Create a list of all the dates across different data sets-

all_dates=[]
for df in [dm_raw,ae_raw,lb_raw,ex_raw,visits_raw]:
    all_dates+=collect_dates(df)

    

In [16]:
# Derive RFSDTC and RFENDTC by taking min and max of all dates list( We usually take these two from DM domain)

minmax=pd.DataFrame(all_dates)
minmax=minmax.groupby("SUBJID")["DATE"].agg(["min","max"]).reset_index()
minmax=minmax.rename(columns={"min":"RFSDTC","max":"RFENDTC"})
subjects=subjects.merge(minmax,on="SUBJID", how="left")


In [18]:
# Similarly Derive RFXSDTC and RFXENDTC

ex_subj=first_col(ex_raw,["subjid","subjectid","subjno","subid"])
ex_start=first_col(ex_raw,["exsdtc","rfexsdtc","start","exstdtc","exstdt"])
ex_end=first_col(ex_raw,["exendtc","rfexendtc","rfexendt","end","exendt"])

keep_cols=[ex_subj,ex_start,ex_end]

ex=ex_raw[keep_cols]

In [19]:
ex_sorted=ex.groupby("SUBJID").agg({"EXSTDTC":"first","EXENDTC":"last"}).reset_index()

In [20]:
subjects=subjects.merge(ex_sorted,on="SUBJID",how="left")

In [21]:
subjects.rename(columns={"EXSTDTC":"RFXSDTC","EXENDTC":"RFXENDTC"},inplace=True)

In [22]:
# Now lets define lb columns which are supposed to searched and then mapped-

lb_subj=first_col(lb_raw,["subjid","subjno","subjectid","subjectnumber"])
lb_orres=first_col(lb_raw,["lborres","orres"])
lb_orresu=first_col(lb_raw,["lborresu","orresu"])
lb_ornrlo=first_col(lb_raw,["lbornrlo","ornrlo"])
lb_ornrhi=first_col(lb_raw,["lbornrhi","ornrhi"])
lb_test=first_col(lb_raw,["lbtest","test"])
lb_testcd=first_col(lb_raw,["lbtestcd","testcd"])
lb_date=first_col(lb_raw,["lbdtc","lbdt","lbdate"])


In [52]:
# Now we will assign original results to the lab data and create a convertor to derive lab data values-

lb=lb_raw.copy()
lb["SUBJID"]=lb[lb_subj]
lb["USUBJID"]= STUDYID+"-"+ lb[lb_subj]
lb["LBORRES"]=lb[lb_orres]
lb["LBORRESU"]=lb[lb_orresu]
lb["LBORNRLO"]=lb[lb_ornrlo]
lb["LBORNRHI"]=lb[lb_ornrhi]
lb["LBTESTCD"]=lb[lb_testcd]
lb["LBDTC"]=lb[lb_date]

In [24]:
# # ---------- Unit conversion setup (simple, safe) ----------
# Use lowercase keys for comparison; add only clear, unique mappings

convert_unit={("mEq/L","mmol/L"):lambda v:v*1,
              # ===== ELECTROLYTE CONVERSIONS (mEq/L ↔ mmol/L) =====
              ("mEq/L", "mmol/L"): lambda v: v * 1,  # For monovalent ions
              ("mmol/L", "mEq/L"): lambda v: v * 1,  # Reverse conversion
              # Special cases for divalent ions (Calcium, Magnesium)
                ("mEq/L", "mmol/L"): lambda v: v / 2,  # For divalent ions
                ("mmol/L", "mEq/L"): lambda v: v * 2,  # Reverse for divalent ions
    
                # ===== MASS CONVERSIONS =====
                ("µg/dL", "g/L"): lambda v: v * 0.00001,
                ("g/L", "µg/dL"): lambda v: v / 0.00001,
    
                ("µmol/L", "g/L"): lambda v: v * 0.0665,
                ("g/L", "µmol/L"): lambda v: v / 0.0665,
    
                ("g/dL", "g/L"): lambda v: v * 10,
                ("g/L", "g/dL"): lambda v: v / 10,
    
                ("G/DL", "g/L"): lambda v: v * 10,
                ("g/L", "G/DL"): lambda v: v / 10}
    
 
preffered_unit={# Preferred Units Dictionary for Clinical Laboratory Tests
    "WBC": "10^9/L",          # White Blood Cell Count
    "RBC": "10^12/L",         # Red Blood Cell Count
    "HGB": "g/dL",            # Hemoglobin
    "HCT": "%",               # Hematocrit
    "MCV": "fL",              # Mean Corpuscular Volume
    "MCH": "pg",              # Mean Corpuscular Hemoglobin
    "MCHC": "g/dL",           # Mean Corpuscular Hemoglobin Concentration
    "PLAT": "10^9/L",         # Platelet Count
    "RDW": "%",               # Red Cell Distribution Width
    "RETI": "%",              # Reticulocyte Count
    "RETIRBC": "ratio",       # Reticulocyte to RBC ratio
    
    # ===== DIFFERENTIAL =====
    "NEUT": "10^9/L",         # Neutrophils
    "LYM": "10^9/L",          # Lymphocytes
    "MONO": "10^9/L",         # Monocytes
    "EOS": "10^9/L",          # Eosinophils
    "BASO": "10^9/L",         # Basophils
    "NEUTLE": "%",            # Neutrophils percentage
    "LYMLE": "%",             # Lymphocytes percentage
    "MONOLE": "%",            # Monocytes percentage
    "EOSLE": "%",             # Eosinophils percentage
    "BASOLE": "%",            # Basophils percentage
    
    # ===== CHEMISTRY - BASIC METABOLIC PANEL =====
    "SODIUM": "mmol/L",       # Sodium
    "K": "mmol/L",            # Potassium
    "CL": "mmol/L",           # Chloride
    "CO2": "mmol/L",          # Bicarbonate/Carbon Dioxide
    "BICARB": "mmol/L",       # Bicarbonate (same as CO2)
    "BUN": "mg/dL",           # Blood Urea Nitrogen
    "CREAT": "mg/dL",         # Creatinine
}


In [25]:
#Convert numeric value 'val' from unit 'from_u' to 'to_u' when mapping exists.Returns tuple (numeric_value_in_target, target_unit)
#Keeps it simple: if conversion rule doesn't exist, returns numeric original + to_u.

def conv_num(val,from_u,to_u):
    fu=str(from_u).lower() if not pd.isna(from_u) else ""
    tu=str(to_u).lower() if not pd.isna(to_u) else ""
    v=try_num(val)
    if pd.isna(v):
        return (np.nan,tu)
    
    fn=convert_unit.get((fu,tu))
    if fn:
        return (fn(val),tu)
    else:
        return (v,tu)
    
    

In [26]:
#Pick preferred / target unit for this row based on LBTESTCD. If a preferred unit exists for the TESTCD, use that; otherwise keep original unit.
def pick_target(row):
    t=row.get("LBTESTCD")
    preff=preffered_unit.get(t)
    return preff if not pd.isna(preff) else row.get("LBORRESU")
    

In [27]:
# assign TARGET_U
lb["TARGET_U"]=lb.apply(pick_target,axis=1)

In [28]:
#Convert original result into standard numeric (LBSTRESN), standard unit (LBSTRESU),and also keep character copy (LBSTRESC) per CDISC guidance.
#Returns a Series with LBSTRESN, LBSTRESU, LBSTRESC.

def convert_val(row):
    val=row.get("LBORRES")
    from_u=row.get("LBORRESU")
    to_u= row.get("TARGET_U")
    n,unit=conv_num(val,from_u,to_u)
    strec=str(val)
    return pd.Series({"LBSTRESN":n,"LBSTRESU":unit ,"LBSTRESC":strec}) 

In [29]:
conv=lb.apply(convert_val,axis=1)

In [30]:
conv.isnull().sum()

LBSTRESN    5081
LBSTRESU       0
LBSTRESC       0
dtype: int64

In [31]:
lb=pd.concat([lb,conv],axis=1)

In [32]:
# Similarly We will convert Ranges-

def nri_safe(row):
    lo=row.get("LBORNRLO") 
    hi=row.get("LBORNRHI")
    fu=row.get("LBORRESU")
    tu=row.get("TARGET_U")
    lo_n,_=conv_num(lo,fu,tu) if pd.notna(lo) else (np.nan,tu)
    hi_n,_=conv_num(hi,fu,tu) if pd.notna(hi) else (np.nan,tu)

    return pd.Series({"LBSTNRLO":lo_n,"LBSTNRHI":hi_n})



In [33]:
# ---------- Derive LBNRIND (normal range indicator: H/L) ----------
# Compare numeric standard result LBSTRESN to LBSTNRLO/LBSTNRHI and return:'H' if above upper, 'L' if below lower, else pd.NA
def nrind(row):
    n=row.get("LBSTRESN")
    lo=row.get("LBSTNRLO")
    hi=row.get("LBSTNRHI")

    try:
        n=float(n) if pd.notna(n) else np.nan
        lo=float(lo) if pd.notna(lo) else np.nan
        hi=float(hi) if pd.notna(hi) else np.nan
    except:
        return np.nan

    # Simple Conversion-

    if pd.isna(n):
        return np.nan
    if pd.notna(hi) and n>hi:
        return "H"
    if pd.notna(lo) and n<lo:
        return "L"
    return np.nan
    

In [34]:
lb["LBNRIND"]=lb.apply(nrind,axis=1)

In [35]:
conv_range=lb.apply(nri_safe,axis=1)

In [36]:
lb=pd.concat([lb,conv_range],axis=1)

In [ ]:
visits_raw

In [37]:
# Derive visit day , visit number andvisit date from visit dataset (This makes sure that lb dates are in proper sync with visit dates)

if visits_raw is not None:
    visits_subj=first_col(visits_raw,["subjid","subjectno","subid"]) 
    visit_num=first_col(visits_raw,["visitnum","visitnumber"])
    visit_day=first_col(visits_raw,["visitday","visitsday","visitdy"])
    visit_start=first_col(visits_raw,["svstdtc","svstdt","subjectvisitstartdate"])
    visit_end=first_col(visits_raw,["svendtc","svend"])
    visit_cols=[visits_subj,visit_num,visit_day,visit_start,visit_end]
    visit=visits_raw[visit_cols]

In [38]:
visit=visit.rename(columns={visits_subj:"SUBJID",visit_num:"VISITNUM",visit_day:"VISITDY",visit_start:"VISIT_DATE"})

In [39]:
visit["VISIT_DATE"]=pd.to_datetime(visit["VISIT_DATE"],errors="coerce").dt.floor("d")

In [40]:
# Get lb date-day version to merge with -

lb["LB_dt"]=pd.to_datetime(lb["LBDTC"],errors="coerce").dt.floor("d")

In [41]:
lb=lb.merge(visit,left_on=["SUBJID","LB_dt"],right_on=["SUBJID","VISIT_DATE"],how="left").reset_index()


In [42]:
lb=lb.merge(subjects,on="SUBJID",how="left")

In [44]:
# Derive Baseline Flag-# ---------- Baseline flag (LBLOBXFL) following SAS-style index approach ----------
# Definition (CDISC): last non-missing value prior to exposure start (RFXSTDTC) -> mark 'Y'
# Steps:
# 1. Keep only rows with LBORRES non-missing and LBDTC_dt and RFXSTDTC present and LBDTC_dt < RFXSTDTC
# 2. For each SUBJID+LBTESTCD choose the row with max LBDTC_dt (i.e., last pre-exposure)
# 3. Mark those rows in LB with LBLOBXFL = 'Y'

eligible=lb[(pd.notna(lb["LBORRES"])) & (pd.notna(lb["LBDTC"]))&(pd.notna(lb["RFXSDTC"])) & (lb["LBDTC"]<lb["RFXSDTC"])].copy()
    

In [45]:
baseline=eligible.groupby("SUBJID")["LBDTC"].idxmax().dropna()

In [46]:
series_list= baseline.astype(int).to_list()

In [47]:
lb.loc[series_list,"LBLOBXFL"]="Y"

In [48]:
lb=lb.sort_values(["SUBJID","LBTESTCD","LBDTC"]).reset_index(drop=True)  #lbcat , lbscat...visitnum, 
lb["LBSEQ"]=lb.groupby("SUBJID").cumcount()+1

In [54]:
#Sample Output-

lb_out = pd.DataFrame({
    "STUDYID": STUDYID,
    "DOMAIN": "LB",
    "USUBJID": lb["USUBJID"],               # study + subject id
    "SUBJID": lb["SUBJID"],
    "LBSEQ": lb["LBSEQ"],                   # sequence number per subject
    "LBTESTCD": lb.get("LBTESTCD", pd.NA),  # short code for test
    "LBTEST": lb.get("LBTEST", pd.NA),      # long test name (if available)
    "LBDTC": lb.get("LBDTC", pd.NA),        # original datetime string
    "LBDY": lb["LBDTC_dt"].apply(lambda x: (x - pd.to_datetime(lb["RFSTDTC"].iloc[0], errors="coerce")).days + 1 if pd.notna(x) and pd.notna(lb["RFSTDTC"].iloc[0]) else pd.NA) if "RFSTDTC" in lb.columns else pd.NA,
    "LBORRES": lb.get("LBORRES", pd.NA),
    "LBORRESU": lb.get("LBORRESU", pd.NA),
    "LBSTRESC": lb.get("LBSTRESC", pd.NA),
    "LBSTRESN": lb.get("LBSTRESN", pd.NA),
    "LBSTRESU": lb.get("LBSTRESU", pd.NA),
    "LBORNRLO": lb.get("LBORNRLO", pd.NA),
    "LBORNRHI": lb.get("LBORNRHI", pd.NA),
    "LBSTNRLO": lb.get("LBSTNRLO", pd.NA),
    "LBSTNRHI": lb.get("LBSTNRHI", pd.NA),
    "LBNRIND": lb.get("LBNRIND", pd.NA),
    "VISIT": lb.get("VISIT", pd.NA),
    "VISITNUM": lb.get("VISITNUM", pd.NA),
    "LBLOBXFL": lb.get("LBLOBXFL", pd.NA),
    "LBBLFL": lb.get("LBBLFL", pd.NA),})


In [55]:
lb_out.to_csv(LB_OUT_PATH,index=False)